I'm having difficultly replicating the results of my TF trained model in pytorch. I'm just gonna manually copy the weights over. 

In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
import h5py
from os import path

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
import tensorflow as tf

/home/sean/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sean/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sean/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sean/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWa

In [5]:
from uatu.scattering import *

In [6]:
#tf_model_name =  '../networks/gupta_net_kappa-28900'
#tf_model_name = '../networks/gupta_bayes_net_kappa-45000'
tf_model_name = '../networks/gupta_net_kappa_adv_abs-9050'

In [7]:
mode = 0
max_order = 0

J = 0
L = 8
K = int(1 + L*J +(L**2)*(J*(J-1))/2.0)

width = 2
smooth = 0
shape = (256, 256)

In [8]:
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

In [9]:
scattering = lambda x: x

In [10]:
try:
    del model
except:
    pass

model = GuptaNet(1)

In [11]:
model.eval()

GuptaNet(
  (layer_0): BasicBlock(
    (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): LeakyReLU(negative_slope=0.01, inplace=True)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (downsample): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (layer_1): BasicBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): LeakyReLU(negative_slope=0.01, inplace=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (downsample): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (layer_2): BasicBlock(
    (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): LeakyReLU(negative_slope=0.01, inplace=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (downsample): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (relu): LeakyReLU(negative_slope=0.01, inplace=True)
  (fc

In [12]:
tf_path = path.abspath(tf_model_name)  # Path to our TensorFlow checkpoint
tf_vars = tf.train.list_variables(tf_path)

In [13]:
var_dict = {}
for var in tf_vars:
    if 'Adam' not in var[0] and 'beta' not in var[0]:
        print(var)
        val =  tf.train.load_variable(tf_path, var[0]).T
        if len(val.shape)>2:
            val = np.swapaxes(val, 2,3)
        #elif len(val.shape)==2:
        #    val = val.T
        var_dict[var[0]] = torch.nn.Parameter(torch.Tensor(val))

('kappa_filters/conv1/weights', [3, 3, 1, 32])
('kappa_filters/conv2/weights', [3, 3, 32, 64])
('kappa_filters/conv3/weights', [3, 3, 64, 128])
('kappa_filters/conv4/weights', [3, 3, 128, 128])
('kappa_filters/conv5/weights', [3, 3, 128, 128])
('kappa_filters/conv6/weights', [3, 3, 128, 128])
('kappa_filters/dense1/biases', [256])
('kappa_filters/dense1/weights', [131072, 256])
('kappa_filters/dense2/biases', [256])
('kappa_filters/dense2/weights', [256, 256])
('kappa_filters/dense3/biases', [2])
('kappa_filters/dense3/weights', [256, 2])


In [14]:
for layer_no in range(3):
    pt_layer = getattr(model, 'layer_%d'%layer_no)
    conv_key =  'kappa_filters/conv%d/weights'%(layer_no*2+1)
    print(conv_key)
    print(pt_layer.conv1.weight.shape)
    pt_layer.conv1.weight = var_dict[conv_key]
    print(pt_layer.conv1.weight.shape)
    #print()
    #bias_key = 'kappa_filters/conv2d/bias' if layer_no == 0 else 'kappa_filters/conv2d_%d/bias'%(layer_no*2)
    print(pt_layer.conv1.bias.shape)
    pt_layer.conv1.bias = torch.nn.Parameter(torch.zeros(pt_layer.conv1.bias.shape[0]))#var_dict[bias_key]
    print(pt_layer.conv1.bias.shape)
    #print()
    
    conv_key = 'kappa_filters/conv%d/weights'%(layer_no*2+2)
    print(conv_key)
    print(pt_layer.conv2.weight.shape)
    pt_layer.conv2.weight = var_dict[conv_key]
    print(pt_layer.conv2.weight.shape)
    #print()
    #bias_key = 'kappa_filters/conv2d_%d/bias'%(layer_no*2+1)
    print(pt_layer.conv2.bias.shape)
    #pt_layer.conv2.bias = var_dict[bias_key]
    pt_layer.conv2.bias = torch.nn.Parameter(torch.zeros(pt_layer.conv2.bias.shape[0]))#var_dict[bias_key]
    print(pt_layer.conv2.bias.shape)
    #print()
    #print('*'*30)
    
for layer_no in range(3):
    pt_layer = getattr(model, 'fc%d'%(layer_no+1))
    print(pt_layer)
    weight_key =  'kappa_filters/dense%d/weights'%(layer_no+1) 
    print(weight_key)
    print(pt_layer.weight.shape)
    pt_layer.weight = var_dict[weight_key]                   
    print(pt_layer.weight.shape)
    bias_key = 'kappa_filters/dense%d/biases'%(layer_no+1) 
    print(bias_key)
    print(pt_layer.bias.shape)
    pt_layer.bias = var_dict[bias_key]                   
    print(pt_layer.bias.shape)


kappa_filters/conv1/weights
torch.Size([32, 1, 3, 3])
torch.Size([32, 1, 3, 3])
torch.Size([32])
torch.Size([32])
kappa_filters/conv2/weights
torch.Size([64, 32, 3, 3])
torch.Size([64, 32, 3, 3])
torch.Size([64])
torch.Size([64])
kappa_filters/conv3/weights
torch.Size([128, 64, 3, 3])
torch.Size([128, 64, 3, 3])
torch.Size([128])
torch.Size([128])
kappa_filters/conv4/weights
torch.Size([128, 128, 3, 3])
torch.Size([128, 128, 3, 3])
torch.Size([128])
torch.Size([128])
kappa_filters/conv5/weights
torch.Size([128, 128, 3, 3])
torch.Size([128, 128, 3, 3])
torch.Size([128])
torch.Size([128])
kappa_filters/conv6/weights
torch.Size([128, 128, 3, 3])
torch.Size([128, 128, 3, 3])
torch.Size([128])
torch.Size([128])
Linear(in_features=131072, out_features=256, bias=True)
kappa_filters/dense1/weights
torch.Size([256, 131072])
torch.Size([256, 131072])
kappa_filters/dense1/biases
torch.Size([256])
torch.Size([256])
Linear(in_features=256, out_features=256, bias=True)
kappa_filters/dense2/weights
t

In [15]:
torch.save(model.state_dict(), '../networks/gupta_net_tf_adv_clone.pth')

In [16]:
model.to(device)

GuptaNet(
  (layer_0): BasicBlock(
    (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): LeakyReLU(negative_slope=0.01, inplace=True)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (downsample): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (layer_1): BasicBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): LeakyReLU(negative_slope=0.01, inplace=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (downsample): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (layer_2): BasicBlock(
    (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): LeakyReLU(negative_slope=0.01, inplace=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (downsample): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (relu): LeakyReLU(negative_slope=0.01, inplace=True)
  (fc

In [17]:
def test(model, x, y, cost_fn):
    x, y = torch.Tensor(x).to(device), torch.Tensor(y).to(device)
    output = model(x)
    #loss = cost_fn(output, y) 
    
    return output.to('cpu').detach().numpy()#, loss.to('cpu').detach().numpy()

In [18]:
f = h5py.File('../data/UatuLightconeTraining.hdf5', 'r')
mean, std =  f.attrs['mean'], f.attrs['std']
f.close()

In [19]:
f = h5py.File('../data/UatuLightconeSameSeed.hdf5', 'r')
boxno = 0
test_X = np.array(f['Box%03d'%boxno]["X"][()])
test_Y = f['Box%03d'%boxno]['Y'][0].reshape((1,2))
f.close()

In [20]:
test_X = (test_X-mean)/(std)

In [21]:
n = 100 
batch_size = 1 

X = test_X[n, :, :, :].reshape((1, test_X.shape[1],\
                                test_X.shape[2], test_X.shape[3]))
X = np.tile(X, (batch_size, 1, 1, 1) )
Y = test_Y[0, :].reshape((1, test_Y.shape[1]))
Y = np.tile(Y, (batch_size, 1))

In [22]:
pred = test(model, X,Y, cost_fn=lambda x,y: torch.nn.functional.l1_loss(x,y, reduction='sum'))

In [23]:
pred

array([[0.28403902, 0.85110223]], dtype=float32)